### Problem Statement:
Assume the market has such a condition that buying today prevent selling the same stock today and tomorrow, but allow selling the day after tomorrow. Conduct only long strategy.<br>

For example: If you buy 100 unit at 1 Oct, you are allowed to sell that after 2 Oct. But you are not allowed to sell it at 1 or 2 Oct.

### Approach
- I've bacteseted Apple stock data from 2011 to 2014 and applied dual moving average crossover strategy with 2 days and 10 days moving average.<br>
- I've skipped 200 days at first as there are some sequential dates after 200 days otherwise I would skip 10 days as the long moving avg.<br>
- I've taken lower moving average to gererate very frequent crossover sothat I get some sequential dates of buy and sell.<br>
- There are two bactesting, one is with the sequential strategy and another is without it to compare whether our strategy is working or not.
- I didn't add any benchmark as the goal was to apply the sequential buy-sell strategy and check whether it works or not simply.

### Import necessary libraries and modules

In [2]:
import pandas as pd
import os
%load_ext zipline
from zipline.api import symbol, order,record, order_target,order_target_percent,get_open_orders
from zipline.utils.calendar_utils import get_calendar
from zipline import run_algorithm
import matplotlib.pyplot as plt
import numpy as np
from zipline.data import bundles
%matplotlib inline
import matplotlib.pyplot as plt
import quantstats as qs
from pyfolio.utils import extract_rets_pos_txn_from_zipline

### 1. Bactest without sequential buy-sell Condition

In [3]:
%%capture
calendar = get_calendar('XNYS')
capital_base = 30000

def initialize(context):
    context.i = 0
    context.asset = symbol('AAPL')

def handle_data(context, data):
    context.i += 1
    if context.i < 200: return

    short_mavg = data.history(context.asset, 'price', bar_count=2, frequency="1d").mean()
    long_mavg = data.history(context.asset, 'price', bar_count=10, frequency="1d").mean()
    
    if not get_open_orders(context.asset):
        if short_mavg > long_mavg:
            order_target(context.asset, 100)

        elif short_mavg < long_mavg:
            order_target(context.asset, 0)

start_date = pd.to_datetime('2011-01-02 00:00:00.0000',format='%Y-%m-%d %H:%M:%S.%f')
end_date = pd.to_datetime('2015-01-01 00:00:00.0000',format='%Y-%m-%d %H:%M:%S.%f')

res = run_algorithm(
    start = start_date,
    end = end_date,
    trading_calendar=calendar,
    initialize=initialize,
    handle_data = handle_data,
    bundle='snp5',
    capital_base=capital_base,
    data_frequency='daily')

### Find some sequential dates of buy and sell signals

In [4]:
trans_non_conditional_list = []

for idx, row in res.iterrows():
    for transaction in row.transactions:
        trans_non_conditional_list.append(transaction)

trans_non_conditional_df = pd.DataFrame(trans_non_conditional_list)
trans_non_conditional_df.drop(["commission","order_id"], axis=1, inplace=True)
trans_non_conditional_df['dt'] = pd.to_datetime(trans_non_conditional_df['dt']).dt.date

print(trans_non_conditional_df.head(15).to_string())

    amount          dt      price               sid
0      100  2011-10-18  15.087540  Equity(0 [AAPL])
1     -100  2011-10-21  14.023985  Equity(0 [AAPL])
2      100  2011-10-31  14.463228  Equity(0 [AAPL])
3     -100  2011-11-03  14.387802  Equity(0 [AAPL])
4      100  2011-11-07  14.283138  Equity(0 [AAPL])
5     -100  2011-11-08  14.500746  Equity(0 [AAPL])
6      100  2011-11-09  14.124059  Equity(0 [AAPL])
7     -100  2011-11-10  13.751121  Equity(0 [AAPL])
8      100  2011-12-01  13.861928  Equity(0 [AAPL])
9     -100  2011-12-15  13.527233  Equity(0 [AAPL])
10     100  2011-12-21  14.166080  Equity(0 [AAPL])
11    -100  2012-03-08  19.347321  Equity(0 [AAPL])
12     100  2012-03-09  19.479735  Equity(0 [AAPL])
13    -100  2012-04-02  22.082953  Equity(0 [AAPL])
14     100  2012-04-03  22.487238  Equity(0 [AAPL])


### Result Analysis
**At index 4 to 7, we can see that we have sequential dates of buy and sell which are from 7 Nov 2011 to 10 Nov 2011**.<br>
**So, we can buy the stock on 7 Nov 2011 and sell it on 9 Nov 2011 or later but not on 8 Nov 2011.**<br>
**We can apply sequential buy-sell condition here.**

**Let's generate strategy with sequential buy-sell condition not to sell the stock on the same day or the next day when I already have position of this stock.**

### Conditional buy-sell strategy not to sell on the same day or next day

In [76]:
%%capture
calendar = get_calendar('XNYS')
capital_base = 30000

def initialize(context):
    context.i = 0
    context.asset = symbol('AAPL')

def handle_data(context, data):
    context.i += 1
    if context.i < 200: return

    short_mavg = data.history(context.asset, 'price', bar_count=2, frequency="1d").mean()
    long_mavg = data.history(context.asset, 'price', bar_count=10, frequency="1d").mean()
    
    current_date = context.get_datetime()
    # print(f"iteration: {context.i}")
    # print(f"current_date: {current_date}")
    
    if not get_open_orders(context.asset):
        if context.blotter.orders:
            last_amount = list(context.blotter.orders.values())[-1].amount  # last order amount
            last_date = list(context.blotter.orders.values())[-1].dt        # last succesful order date
            date_diff = current_date - last_date
            # print(f"last_amount: {last_amount}")
            # print(f"last_dt: {last_date}")
            # print(f"date_diff: {date_diff}")
            
            if date_diff.days < 2 and last_amount > 0: # if last order was a buy and there is a sequence
                if short_mavg > long_mavg:
                    order_target(context.asset, 100)
            
            elif short_mavg > long_mavg:
                order_target(context.asset, 100)
            elif short_mavg < long_mavg:
                order_target(context.asset, 0)
                
        else:
            if short_mavg > long_mavg:
                order_target(context.asset, 100)

start_date = pd.to_datetime('2011-01-02 00:00:00.0000',format='%Y-%m-%d %H:%M:%S.%f')
end_date = pd.to_datetime('2015-01-01 00:00:00.0000',format='%Y-%m-%d %H:%M:%S.%f')

res_conditional = run_algorithm(
    start = start_date,
    end = end_date,
    trading_calendar=calendar,
    initialize=initialize,
    handle_data = handle_data,
    bundle='snp5',
    capital_base=capital_base,
    data_frequency='daily')

### Find any sequential dates of buy and sell signals

In [77]:
trans_conditional_list = []

for idx, row in res_conditional.iterrows():
    for transaction in row.transactions:
        trans_conditional_list.append(transaction)

trans_conditional_df = pd.DataFrame(trans_conditional_list)
trans_conditional_df.drop(["commission","order_id"], axis=1, inplace=True)
trans_conditional_df['dt'] = pd.to_datetime(trans_conditional_df['dt']).dt.date

print(trans_conditional_df.head(20).to_string())

    amount          dt      price               sid
0      100  2011-10-18  15.087540  Equity(0 [AAPL])
1     -100  2011-10-21  14.023985  Equity(0 [AAPL])
2      100  2011-10-31  14.463228  Equity(0 [AAPL])
3     -100  2011-11-03  14.387802  Equity(0 [AAPL])
4      100  2011-11-07  14.283138  Equity(0 [AAPL])
5     -100  2011-11-10  13.751121  Equity(0 [AAPL])
6      100  2011-12-01  13.861928  Equity(0 [AAPL])
7     -100  2011-12-15  13.527233  Equity(0 [AAPL])
8      100  2011-12-21  14.166080  Equity(0 [AAPL])
9     -100  2012-03-08  19.347321  Equity(0 [AAPL])
10     100  2012-03-09  19.479735  Equity(0 [AAPL])
11    -100  2012-04-02  22.082953  Equity(0 [AAPL])
12     100  2012-04-03  22.487238  Equity(0 [AAPL])
13    -100  2012-04-16  20.708641  Equity(0 [AAPL])
14     100  2012-04-27  21.546768  Equity(0 [AAPL])
15    -100  2012-05-02  20.917536  Equity(0 [AAPL])
16     100  2012-05-23  20.387189  Equity(0 [AAPL])
17    -100  2012-06-05  20.090950  Equity(0 [AAPL])
18     100  

### Result Analysis
**There are no sequential dates of buy and sell signals from 7 Nov 2011 to 10 Nov 2011 as before.**<br>
- We can observe that the strategy went to long position on 7 Nov 2011 and sold it on 10 Nov 2011.
- It din't sell it on 8 Nov 2011 as it was the next day of buying.
- It din't sell it on 9 Nov 2011 as it was bulish crossover(short moving avg > long moving avg)
- It sold it on 10 Nov 2011 as it was bearish crossover and not a sequential date of buying.

**That means our strategy is working perfectly.**

### Function for all sequential date counter (Verification of the strategy)
We have checked a single sequence but need to check whether we have sequential dates of buy and sell signals or not for the full order list.<br>
I've created a function to check it.
- It applies a simple algorithm of finding sequential array elements to check whether we have sequential dates of 2 days or not.
- There should be no sequential dates of 2 days for the full order list after applying the sequential buy-sell strategy.
- But there should be sequential dates of 2 days for the strategy without sequential buy-sell strategy. The reason behind that is ```zipline``` transactions keep the successful transactions only in the ```transcations``` pandas series.
- I also checked I'm counting the sequence of lengh 2 only when the first '''amount''' of the sequence is buy and the second '''amount''' is sell. Which is in line 6 of the function. ```df['amount'].iloc[i] < df['amount'].iloc[i-1]```

In [73]:
def count_consequtive_dates(df,length):
    lst = []
    counter = 1
    items = 0
    for i in range(1,len(df)):
        if (df['dt'].iloc[i] - df['dt'].iloc[i-1]).days == 1 and df['amount'].iloc[i] < df['amount'].iloc[i-1]:
            counter += 1
            if counter == length:
                lst.append(df['dt'].iloc[i - 1])
                items += 1
            counter = 1
        else:
            counter = 1
    if counter == length:
        items += 1
        lst.append(df['dt'].iloc[i - 1])
    return {"Sequence_counted":items, "List":lst}

### First Check for the conditioanl buy-sell strategy

In [78]:
print("Number of consequtive length of 2 in conditional_buy_sell:", count_consequtive_dates(trans_conditional_df,2)["Sequence_counted"])

Number of consequtive length of 2 in conditional_buy_sell: 0


### Result Analysis
**Great ! There is no sequential dates in the transactions as expected for our strategy.**<br>
**That means our strategy didn't sell the stock on the same day or the next day when I already have position of this stock.**

### Now Check for the non conditioanl buy-sell strategy

In [79]:
print("Number of consequtive length of 2 in conditional_buy_sell:", count_consequtive_dates(trans_non_conditional_df,2)["Sequence_counted"])

Number of consequtive length of 2 in conditional_buy_sell: 6


In [80]:
if count_consequtive_dates(trans_non_conditional_df,2)["List"]:
    for date in count_consequtive_dates(trans_non_conditional_df,2)["List"]:
        print(date)
else:
    print("No sequences found")

2011-11-07
2011-11-09
2012-10-18
2013-01-24
2013-11-06
2014-03-20


### Result Analysis
There are 6 sequential dates of 2 days in the transactions as expected before applying the sequential buy-sell strategy. And the sequnces are listed above.

### Drawback of the strategy
- I've developed the strategy only when ```order_target``` is used for going long and close positions of the stock.
- If I use ```order``` instead of ```order_target``` then it needs a little modification in the strategy.